In [22]:
import os
import sys
import json
import time
import random
from sklearn.model_selection import train_test_split

In [ ]:
{"edges": [[0, 1],[1, 2],[2, 3],[3, 4]],
 "labels": {"0": "A", "1": "B", "2": "C", "3": "A", "4": "B"},
 "target": 1}

In [186]:
tasks = os.listdir('./data/NCI_txt/')

In [187]:
tasks.remove('P388')
tasks.remove('Yeast')
tasks

['MCF-7',
 'MOLT-4',
 'NCI-H23',
 'OVCAR-8',
 'PC-3',
 'SF-295',
 'SN12C',
 'SW-620',
 'UACC257']

In [188]:
meta_data = {}
first_loop = 0
begin = time.time()

##########
for task in tasks:
    ##########
    edges_raw = open('./data/NCI_txt/%s/%s_A.txt'%(task,task)).read().split('\n')[:-1]
    graph_ID_of_nodes = open('./data/NCI_txt/%s/%s_graph_indicator.txt'%(task,task)).read().split('\n')[:-1]
    node_labels_raw = open('./data/NCI_txt/%s/%s_node_labels_multitask.txt'%(task,task)).read().split('\n')[:-1]
    graph_labels_raw = open('./data/NCI_txt/%s/%s_graph_labels.txt'%(task,task)).read().split('\n')[:-1]
    
    ##########
    track = 0
    duplicate_bin = []
    start_node = 0
    start_edge = 0
    print('Processing %s - %s graphs'%(task,len(graph_labels_raw)))
    ##########
    for i in graph_labels_raw :
        ###### Add graph target
        data = {}
        data['target%s'%tasks.index(task)] = int(i)

        ######
        try:
            num_nodes = graph_ID_of_nodes.index(str(track+2)) - start_node
        except:
            num_nodes = len(graph_ID_of_nodes) - int(graph_ID_of_nodes.index(str(track+1)))

        ######
        nodes_labels = node_labels_raw[start_node:num_nodes+start_node]
        

        ###### Add node labels
        data['labels'] = {'0':nodes_labels[0]}
        nodes_identifier = 1
        for label in nodes_labels[1:]:
            data['labels'][str(nodes_identifier)] = label
            nodes_identifier+=1

        ###### Add edges
        edges = []

        for pair in edges_raw[start_edge:]:
            last = int(pair.split(',')[-1])
            first = int(pair.split(',')[0])

            if last <= (start_node+num_nodes):
                first = first - start_node -1
                last = last - start_node -1
                edges.append([first,last])
            else:
                break

        data['edges'] = edges
        
        compound = [tuple(i) for i in edges]
        compound = hash(tuple(compound))
        
        
        ####### update tracking
        start_node+=num_nodes
        start_edge+=len(edges)
        track+=1
        #######
        if track % 5000 == 0:
            print(track)
            
        ######
        if compound not in duplicate_bin:
            if compound not in meta_data and first_loop == 0:
                meta_data[compound] = [data]
            elif compound not in meta_data and first_loop == 1:
                pass
            else:
                meta_data[compound].append(data)
        else:
            if compound in meta_data:
                del meta_data[compound]
        
        #####
        duplicate_bin.append(compound)
        
    #######
    if tasks.index(task) != 0 :
        for each in list(meta_data):
            if len(meta_data[each]) < (tasks.index(task) + 1):
                del meta_data[each]
    #######
    first_loop = 1
    print('%s done'%task)
    print('Remain graphs: %s\n'%len(meta_data))



###########
outdir = './data/input/train/'
track = 0

for each in meta_data:
    #######
    data = meta_data[each][0]
    
    #######
    for i in range(len(meta_data[each])):
        data['target%s'%i] = meta_data[each][i]['target%s'%i]
    
    #######
    ####### Save file
    outfile = open(os.path.join(outdir,os.path.basename('graph_%s.json'%track)),'w')
    outfile.write(json.dumps(data))  
    outfile.close()
    #######
    track+=1


        
print('Time: %s'%(time.time()-begin))



Processing MCF-7 - 27770 graphs
5000
10000
15000
20000
25000
MCF-7 done
Remain graphs: 26754

Processing MOLT-4 - 39765 graphs
5000
10000
15000
20000
25000
30000
35000
MOLT-4 done
Remain graphs: 25488

Processing NCI-H23 - 40353 graphs
5000
10000
15000
20000
25000
30000
35000
40000
NCI-H23 done
Remain graphs: 24700

Processing OVCAR-8 - 40516 graphs
5000
10000
15000
20000
25000
30000
35000
40000
OVCAR-8 done
Remain graphs: 24117

Processing PC-3 - 27509 graphs
5000
10000
15000
20000
25000
PC-3 done
Remain graphs: 22857

Processing SF-295 - 40271 graphs
5000
10000
15000
20000
25000
30000
35000
40000
SF-295 done
Remain graphs: 22386

Processing SN12C - 40004 graphs
5000
10000
15000
20000
25000
30000
35000
40000
SN12C done
Remain graphs: 21596

Processing SW-620 - 40532 graphs
5000
10000
15000
20000
25000
30000
35000
40000
SW-620 done
Remain graphs: 21071

Processing UACC257 - 39988 graphs
5000
10000
15000
20000
25000
30000
35000
UACC257 done
Remain graphs: 20527

Time: 13834.687901258469

In [10]:
data = os.listdir('./data/input/train_full/')
graphs = []
for each in data:
    graphs.append(json.loads(open('./data/input/train_full/%s'%each).read()))

In [33]:
count = 0
sample = []
count = 0
track = 0
X = []
y = []

ID = [i for i in range(len(graphs))]
random.shuffle(ID)

for i in ID:
    if graphs[i]['target1'] == 1:
        sample.append(graphs[i])
        X.append(track)
        y.append(graphs[i]['target1'])
        track+=1
        
for i in ID:
    if graphs[i]['target1'] == 0:
        sample.append(graphs[i])
        X.append(track)
        y.append(graphs[i]['target1'])
        track+=1
        count+= 1
    if count > 2100 :
        break

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [35]:
###########
track = 0

for each in X_train:
    #######
    data = sample[each]
    outdir = './data/input/train/'
    
    #######
    ####### Save file
    outfile = open(os.path.join(outdir,os.path.basename('graph_%s.json'%track)),'w')
    outfile.write(json.dumps(data))  
    outfile.close()
    #######
    track+=1

for each in X_test:
    #######
    data = sample[each]
    outdir = './data/input/test/'
    
    #######
    ####### Save file
    outfile = open(os.path.join(outdir,os.path.basename('graph_%s.json'%track)),'w')
    outfile.write(json.dumps(data))  
    outfile.close()
    #######
    track+=1